In [1]:
import numpy as np
import tensorflow as tf
import sys
import random
import matplotlib.pyplot as plt
from experiments.models import * ## repspace
#%load_ext autoreload
#%autoreload 2

%matplotlib inline


In [8]:
np.random.seed(0)

def rbf_kernel(x, y, s=0.5):
    """ Vectorized Gaussian RBF kernel """
    return np.exp(-((x-y)*(x-y)).sum(axis=1)/(2*s*s))

def mmd_h_(x0, x1, y0, y1, s=0.5):
    """ Implements the one-sample U-statistic from Lemma 6 in A kernel two-sample test"""
    
    print(np.max(rbf_kernel(x0, x1, s)))
    print(np.max(rbf_kernel(y0, y1, s)))
    print(np.max(rbf_kernel(x0, y1, s)))
    print(np.max(rbf_kernel(x1, y0, s)))
    return rbf_kernel(x0, x1, s) + rbf_kernel(y0, y1, s) - rbf_kernel(x0, y1, s) - rbf_kernel(x1, y0, s)

def mmd_lin(x, y, s=0.5):
    """ Linear MMD test statistic """
    N1=x.shape[0]
    N2=y.shape[0]
    if N1==N2:
        #carry on
        n = N1
        n2 = int(n/2)
        x0 = x[:n2,]
        x1 = x[n2:(2*n2),]
        y0 = y[:n2,]
        y1 = y[n2:(2*n2),]
    elif N1>N2:
        # take N2 
        n = N2
        n2 = int(n/2)
        x0 = x[:n2,]
        x1 = x[n2:(2*n2),]
        y0 = y[:n2,]
        y1 = y[n2:(2*n2),]
    else:
        # take N1
        n = N1
        n2 = int(n/2)
        x0 = x[:n2,]
        x1 = x[n2:(2*n2),]
        y0 = y[:n2,]
        y1 = y[n2:(2*n2),]
        
    #print(np.max(mmd_h_(x0, x1, y0, y1, s)))
    return mmd_h_(x0, x1, y0, y1, s).mean()


def repspace_mmd(model,x_source,x_target):
    """
    A function which estimates the kernel MMD in the representation space of the classifier. 
    I.e, the function takes each datapoint and runs it through the network until the last layer before the classification is done.
    There, it takes the result and computes the linear mmd between source and target.
    """
    print(model.get_layer(index = 7))
    partial_model = tf.keras.Model(model.input, model.get_layer(index = 7).output ) # the new model which outputs the layer we want
    
    ## convert to rep space
    rep_source=partial_model.predict(x_source)
    rep_target=partial_model.predict(x_target)
    ### compute mmd and return
    N = 10#
    result=[]
    sigmas=[1e-4,1e-3,1e-2,1e-1,1,1e2,1e3,1e4]
    for sigma in sigmas:
        ms=[]
        for i in range(N):
            ## proxy for randomly selecting datapoints in the mmd linear function
            random.shuffle(rep_source)
            random.shuffle(rep_target)
            ms.append(mmd_lin(rep_source, rep_target, sigma))
        result.append(np.mean(ms))
    return result
def flatten_images(x):
    N=len(x)
    return x.reshape(N,-1)


from data.tasks import *
task=6
x_source,_,x_target,_,_,_=load_task(task)

## set the project folder to something for saving
#project_folder="/cephyr/users/adambre/Alvis/mnist_transfer/"
project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"
alpha=0.3
seed=3
arch="lenet"

model=init_task_model(task,True,arch,prior_weights=None)
prior_path=project_folder+"priors/"+"task"+str(task)+"/Binary/"+str(arch)+"/"+"32_"+str(int(100*alpha))+"_"+str(int(seed))+"/prior.ckpt"
model.load_weights(prior_path)
result=repspace_mmd(model,x_source,x_target)
print(result)
#sys.exit(-1)




print(x_source.shape)
x_source=flatten_images(x_source)
x_target=flatten_images(x_target)
print(x_source.shape)
ms = []
#n = 5000
N = 10#0000
result=[]
sigmas=[1e-4,1e-3,1e-2,1e-1,1,1e2,1e3,1e4]
for sigma in sigmas:
    ms=[]
    for i in range(N):
        ## proxy for randomly selecting datapoints in the mmd linear function
        random.shuffle(x_source)
        random.shuffle(x_target)
        ms.append(mmd_lin(x_source, x_target, sigma))
    result.append(np.mean(ms))
    
    
#plt.hist(ms, bins=20)
plt.semilogx(sigmas,result)
plt.show()

print(np.sqrt(np.max(result)))

amount of chexpert examples 223648
3.992669046011382
4.006303392220287
length of chestxray in source 22424
amount of chest in source and target 4.0
mean of weights in source, should be 1:  1.0000000000000002
source 246072
[211596  34476]
----------------------------------------
target 89696
[41392 48304]
1.0
1.0
0.97799677
0.9947261
1.0
1.0
0.97994995
0.91958994
1.0
1.0
0.9951795
0.8124978
1.0
1.0
0.99008185
0.99991393
1.0
1.0
0.98789823
0.9634473
1.0
1.0
0.8394952
0.89770865
1.0
1.0
0.8394952
0.95639515
1.0
1.0
0.95639515
0.8394952
1.0
1.0
0.8394952
0.007118853
1.0
1.0
0.35413045
0.8394952
1.0
1.0
0.77921623
0.77921623
1.0
1.0
0.6130651
0.77921623
1.0
1.0
0.99825203
0.6130651
1.0
1.0
0.6130651
0.77921623
1.0
1.0
0.5328313
0.5328313
1.0
1.0
0.1391525
0.1391525
1.0
1.0
0.0003394621
0.1391525
1.0
1.0
0.0003394621
0.1391525
1.0
1.0
0.0003394621
0.0
1.0
1.0
0.0
0.0
1.0
1.0
7.1295555e-37
7.1295555e-37
1.0
1.0
7.1295555e-37
7.1295555e-37
1.0
1.0
0.0
7.1295555e-37
1.0
1.0
0.0
7.1295555e-37
1.

KeyboardInterrupt: 